In [ ]:
import pandas as pd
import numpy as np
import datetime

# Chargement des données de la base de donnée des crimes, de 2001 à 2022 à Chicago

La base de donnée est très lourde : 7 531 858. Il a été impossible de la lire en une seule fois. 

En la lisant seulement avec un **read_csv** classique, le noyau meurt systématiquement. En utilisant les **chunksize**, la RAM de l'ordinateur est arrivée à bout.

Comme nous n'avons pas de base de données sur les écoles pour chaque année renseignée dans la base de donnée des crimes, l'objectif est de lire seulement les années nécessaires.

Il faut donc déterminer les **rows** inutile à lire. Pour se faire, nous commençons par lire seulement la colonne des dates de la BD.

In [ ]:
print("Starting...")
crimes_url = "data/Crimes_-_2001_to_Present.csv"
crimes_df = pd.read_csv(crimes_url, parse_dates=[0], usecols=[2])
print("End...")

print(len(crimes_df))

In [ ]:
print(crimes_df.dtypes)

Nous fixons maintenant un seuil de dates pour déterminer quels sont les **rows** à garder ou non.

In [ ]:
date_inf = np.datetime64('2015-01-01')
date_sup = np.datetime64('2017-01-01')
def cond(i):
    return (crimes_df.iloc[i]['Date'] >= date_inf and crimes_df.iloc[i]['Date'] < date_sup)

Nous enregistrons les index des lignes pour lesquelles la date est en dehors du seuil fixé. Ces lignes n'ont pas besoin d'être lues.

In [ ]:
invalid_dates = []
for i in range (len(crimes_df) - 1):
    if not cond(i):
        invalid_dates.append(i)
len(invalid_dates)

On lit notre base de donnée sur toutes les colonnes mais en ignorant les lignes déterminées précedemment.

In [ ]:
print("Starting...")
print("Starting...")
chunksize = 10000
list_of_df = []
for df in pd.read_csv(crimes_url, skiprows=invalid_dates, chunksize=chunksize):
    list_of_df.append(df)
print(len(list_of_df))
crimes_df = pd.concat(list_of_df)
print("End...")

In [ ]:
print(len(crimes_df))

On enregistre la base de donnée avec toute ses colonnes mais seulement les lignes des années sélectionnées.

In [ ]:
crimes_df.to_csv("data/crimes_2015-2017.csv")

Finalement, on retire les colonnes inutiles avant d'enregister notre BD dans un format csv.

In [ ]:
remove = ['ID', 'Case Number', 'Description', 'Arrest', 'Beat', 'District', 'Community Area', 'FBI Code', 'X Coordinate', 'Y Coordinate', 'Year', 'Updated On', 'Location']
reducted_db = crimes_df.drop(remove, axis=1)
reducted_db.to_csv("data/reductedDB_2015-2017.csv")
reducted_db